In [1]:
import os
os.chdir("/Users/maciej.filanowicz/Desktop/token_classification")

In [2]:
from hydra import initialize, compose
import json
from src.inference import InferencePipeline
from src.utils import visualize_predictions
import torch
from PIL import Image
from transformers import LayoutLMv3Processor

In [3]:
initialize(version_base="1.2", config_path="../conf", job_name="test_app")
cfg  = compose(config_name="config2", return_hydra_config=True)

In [4]:
device = torch.device('cpu')

In [5]:
processor = LayoutLMv3Processor.from_pretrained(cfg.model.pretrained_model_name,
                                                                 apply_ocr=False)

In [6]:
# with open('datasets/FUNSD/annotations/00070353.json','r') as fp:
#     document = json.load(fp)
# document_image = Image.open('datasets/FUNSD/images/00070353.png').convert('RGB')
with open('datasets/FUNSD/annotations/01122115.json','r') as fp:
    document = json.load(fp)
document_image = Image.open('datasets/FUNSD/images/01122115.png').convert('RGB')

In [7]:
inference_pipeline = InferencePipeline(model_path='models/model.pth',
                                       processor=processor,
                                       device=device,
                                       id2label=cfg.dataset.id2label)
processed_output = inference_pipeline.predict([document_image],[document])[0]

/Users/maciej.filanowicz/opt/anaconda3/envs/nlp/lib/python3.9/site-packages/transformers/modeling_utils.py:812: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [9]:
visualize_predictions(document_image,processed_output).save('example.png')